<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>


# LinkedIn - List new connections monthly
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/LinkedIn/Linkedin_Follow_number_of_content_published.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a>

**Tags:** #linkedin #html #plotly #csv #image #content #analytics #automation

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

## Input

### Get common variables, functions

In [ ]:
# Except allow to run common.ipynb once using Naas Engine
try:
    %run "../common.ipynb"
except:
    %run "common.ipynb"

### Setup Variables

In [ ]:
KPI_TITLE = "New connections"

### Setup Constants

In [ ]:
# Inputs
PLATFORM = LINKEDIN
LOGO = LINKEDIN_LOGO
OWNER = LK_FULLNAME
SCENARIO = SCENARIO_MTD
PERIOD = PERIOD_MTD
COL_DATE = "CREATED_AT"

# Input
csv_input = path.join(LINKEDIN_OUTPUTS, f"LINKEDIN_FOLLOW_PROFILE_CONNECTIONS_{SCENARIO}_{LK_PROFILE_ID}.csv")

## Model

### Get connections from LinkedIn network

In [ ]:
df_connections = get_data(LK_PROFILE_CONNECTIONS)
print("Data fetched:", len(df_connections))
df_connections.head(1)

### Get data

In [ ]:
def calc_value(csv_input):
    df = get_data(csv_input)
    if len(df) > 0:
        value = df.loc[df.index[-1], "VARV"]
        value_comp = df.loc[df.index[-2], "VARV"]
        varv = value - value_comp
        varp = varv / abs(value_comp)
        return value, value_comp, varv, varp
    else:
        return 0, 0, 0, 0

value, value_comp, varv, varp = calc_value(csv_input)
# print(value, value_comp, varv, varp)

### Get list

In [ ]:
def get_list(df_init,
              col_date):
    
    # Init variable
    df = df_init.copy()
    df[SCENARIO] = pd.to_datetime(df[col_date]).dt.strftime(PERIOD)
    df["DATE"] = pd.to_datetime(df[col_date]).dt.strftime("%Y-%m-%d")
    df["HOURS"] = pd.to_datetime(df[col_date]).dt.strftime("%H:%M:%S")
    
    # Filter
    df = df[df[SCENARIO] == datetime.now().strftime(PERIOD)].reset_index(drop=True)
    
    # Sort
    df = df.sort_values(by=col_date, ascending=False)
    return df.reset_index(drop=True)

df_new = get_list(df_connections,
                  col_date=COL_DATE)
print("New rows:", len(df_new))
df_new.head(1)

## Output

### Update Notion Report

In [ ]:
def update_report_status(report,
                         platform,
                         owner,
                         df=pd.DataFrame(),
                         image_link=None,
                         html_link=None,
                         csv_link=None,
                         page_icon=LOGO,
                         ):
    # Decode database id
    database_id = naas.secret.get(name="NOTION_DATABASE_URL").split("/")[-1].split("?v=")[0]
    
    # Get pages from notion database
    pages = notion.connect(NOTION_TOKEN).database.query(database_id)
    
    # Create or update page
    page_new = True
    for page in pages:
        page_temp = page.df()
        page_id = page_temp.loc[page_temp.Name == "Report name", "Value"].values
        page_id2 = page_temp.loc[page_temp.Name == "Platform", "Value"].values
        page_id3 = page_temp.loc[page_temp.Name == "Owner", "Value"].values
        if page_id == report and page_id2 == platform and page_id3 == owner:
            page_new = False
            break
    try:
#         notion.connect(NOTION_TOKEN).blocks.delete(page.id)
        if page_new:
            page = notion.connect(NOTION_TOKEN).Page.new(database_id=database_id).create()
            page.title("Report name", report)
            page.multi_select("Platform", [platform])
            page.select("Owner", owner)
            notion.client.pages.update(page_id=page.id, icon={'type': 'external', 'external': {'url': page_icon}})
            
        # Check if image already exists
        blocks = page.get_blocks()
        if len(blocks) > 0:
            while True:
                for block in blocks:
                    notion.connect(NOTION_TOKEN).blocks.delete(block.id)
                blocks = page.get_blocks()
                if len(blocks) == 0:
                    break
        page.update()
        
        dates = df.DATE.unique()
        for d in dates:
            tmp_df = df[df.DATE == d].reset_index(drop=True)
            page.heading_3(d)
            for index, row in tmp_df.iterrows():
                res = page.bulleted_list_item(f"{row.FIRSTNAME} {row.LASTNAME} - {row.OCCUPATION} ({row.HOURS})")
                res.bulleted_list_item.text[0].href = row.PROFILE_URL
                res.bulleted_list_item.text[0].text.link = Link(row.PROFILE_URL)

        # Create page in Notion
        value, value_comp, varv, varp = calc_value(csv_input)
        page.number("This month", float(value))
        page.number("Last month", float(value_comp))
        page.number("Variation", float(varv))
        page.number("Variation %", round(float(varp), 4))
        page.date("Last updated date", datetime.now(pytz.timezone(TIMEZONE)).strftime("%Y-%m-%d %H:%M:%S%z"))
        page.update()
        print(f"✅ Page '{report} - {platform} - {owner}' updated in Notion.")
    except Exception as e:
        raise(e)
    return page

page = update_report_status(KPI_TITLE,
                            PLATFORM,
                            OWNER,
                            df_new)